# Water Quality Monitoring with Sentinel-2 with Dynamic Compute

Here, we explore an example of using Sentinel-2 [Se2WaQ equations](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/se2waq/) to assess water quality over Neuchatel Lake near Bern, Switzerland. 

### Methodology
Originally taken from [Miguel Potes et al, 2018](https://piahs.copernicus.org/articles/380/73/2018/), these equations were derived from empirically tuned models fit on _in situ_ data. These results are meant to demonstrate applying a scientific methodology to earth observation data, and not to act as definitive truth to water quality conditions. 

* **Chlorophyll A** content as mg/m^3:

\begin{equation*}
4.26*(GREEN / COASTAL)^{3.94}
\end{equation*} 

* **Turbidity** as NTU:

\begin{equation*}
8.93*(GREEN / COASTAL)-6.39
\end{equation*} 

### Dynamic Compute

Using [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html), one can quickly iterate on raster analysis methodologies and extend to new geographies on-the-fly. 

The general steps are as follows:
1. Create a [`Mosaic`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic) object referencing Sentinel-2 imagery over the date range in question, alongside the required `band`s
2. [`Unpack`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic.unpack_bands) the necessary bands to input to water quality equations (`coastal`, `green`, and `nir`)
3. Calculate **NDWI** to define an input water/land mask
4. Calculate the relevant Chlorophyll A and Turbidity values with `Dynamic Compute`'s native numpy integration
5. [`Visualize`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic.visualize) results on an interactive [leaflet map](https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/map.html) to extend the analysis to new geographies

_Imports_


In [ ]:
import descarteslabs.dynamic_compute as dc

Define a Sentinel-2 [`Mosaic`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic) over our date range:

In [ ]:
mosaic = dc.Mosaic.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "coastal-aerosol green nir red blue",
    start_datetime="2019-06-28", 
    end_datetime="2019-06-30"
)

[`Unpack`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic.unpack_bands) the coastal, green, and near-infrared bands:

In [ ]:
coastal, green, nir = mosaic.unpack_bands("coastal-aerosol green nir")

Calculate a [Normalized Difference Water Index](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-3/ndwi/#:~:text=The%20NDWI%20is%20used%20to,over%2Destimation%20of%20water%20bodies.) to mask out land:

In [ ]:
ndwi = (nir - green)/(nir+green)

Caluclate Chlorophyll A content:

In [ ]:
chl_a = 4.26*dc.operations.np.power(green/coastal, 3.94)

Calculate Turbidity:

In [ ]:
turb = 8.93*(green/coastal)-6.39

Mask both layers to water:

In [ ]:
chl_a_msk = chl_a.mask(ndwi>0)
turb_msk = turb.mask(ndwi>0)

Create a leaflet map:

In [ ]:
m = dc.map
m.center = 46.9596, 6.9968
m.zoom = 12

Add an RGB basemap from the input Sentinel-2 imagery:

In [ ]:
mosaic.pick_bands("red green blue").visualize("RGB", m)

In [ ]:
chl_a_msk.visualize("Chlorophyll A", m, colormap='jet')
turb_msk.visualize("Turbidity", m, colormap='coolwarm')

In [ ]:
m